In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import duckdb
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str(datetime.now().date() - timedelta(days=1))
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
stats = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

Today's date: 2025-11-24


In [12]:
%run ./common_utils.ipynb

In [14]:
def create_actuals_tbl(con):
    df_lines = pd.read_csv(f"../tables/{year}/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[(df_lines.Date == now)]
    df_log = pd.read_csv(f"../tables/{year}/season_gamelogs.csv")
    df_log['Date'] = pd.to_datetime(df_log.Date)
    df_log = df_log[(df_log.Date == now)]
    df_log = df_log.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
    df_log['PR'] = df_log.PTS + df_log.REB 
    df_log['PA'] = df_log.PTS + df_log.AST
    df_log['RA'] = df_log.REB + df_log.AST
    df_log['PRA'] = df_log.PTS + df_log.REB + df_log.AST
    df_log['STL_BLK'] = df_log.STL + df_log.BLK
    df = con.execute("""SELECT t2.*, t1.* EXCLUDE(Date, Team, Player, Pos) FROM df_log t1 JOIN df_lines t2 
                        ON t1.Team = t2.Team AND t1.Player = t2.Player""").fetchdf()
    cols = []
    for stat in stats:
        df[f'{stat}_Result'] = np.where(df[stat] > df[f'{stat}_line'], 'O', 'U')
        df[f'{stat}_Result'] = np.where(df[f'{stat}_line'].isnull(), np.nan, df[f'{stat}_Result'])
        df[f'{stat}_Diff'] = df[stat] - df[f'{stat}_line']
        cols.append(stat)
        cols.append(f'{stat}_line')
        cols.append(f'{stat}_Diff')
        cols.append(f'{stat}_Result')
    df = df[['game_id', 'Date', 'Team', 'Player', 'Pos', 'MP'] + cols]
    
    df_stats = pd.read_csv(f"../tables/{year}/parlay_stats.csv")
    df_stats['Date'] = pd.to_datetime(df_stats.Date).astype(str)
    lines = []
    diffs = []
    stats_cols = []
    
    for stat in stats:
        lines.append(f'{stat}_line')
        diffs.append(f'{stat}_Diff')
    for stat in stats:
        stats_cols.extend([
            f"Off_{stat}",
            f"Off_L5_{stat}",
            f"AVG_{stat}_H2H",
            f"Def_{stat}",
            f"Def_L5_{stat}",
            f"df_stats.{stat}_line",
            f"{stat}",
            f"df.{stat}_Diff",
            f"{stat}_Result"
        ])
    df_select = ", ".join(stats_cols)
    lines = str(lines).replace("[", "").replace("]", "")
    diffs = str(diffs).replace("[", "").replace("]", "")
    df_actuals = con.execute(f"""SELECT game_id, df.Date, df.Team, B2B, Spread, Total, df.Player, df.Pos, MP, Opp,
                                 {df_select} FROM df JOIN df_stats 
                                 ON df.Date = df_stats.Date AND df.Player = df_stats.Player
                                 """).fetchdf()
    partition_save_df(df_actuals, f"../tables/{year}/parlay_actuals.csv")
    display(df_actuals)

    return df_actuals
                      
df = create_actuals_tbl(con)

../tables/2025/parlay_actuals.csv saved!


,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,PTS,PTS_Diff,PTS_Result,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,AST,AST_Diff,AST_Result,Off_REB,Off_L5_REB,AVG_REB_H2H,Def_REB,Def_L5_REB,REB_line,REB,REB_Diff,REB_Result,Off_PR,Off_L5_PR,AVG_PR_H2H,Def_PR,Def_L5_PR,PR_line,PR,PR_Diff,PR_Result,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,PA,PA_Diff,PA_Result,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,RA,RA_Diff,RA_Result,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,PRA,PRA_Diff,PRA_Result,Off_TPM,Off_L5_TPM,AVG_TPM_H2H,Def_TPM,Def_L5_TPM,TPM_line,TPM,TPM_Diff,TPM_Result,Off_STL,Off_L5_STL,AVG_STL_H2H,Def_STL,Def_L5_STL,STL_line,STL,STL_Diff,STL_Result,Off_BLK,Off_L5_BLK,AVG_BLK_H2H,Def_BLK,Def_L5_BLK,BLK_line,BLK,BLK_Diff,BLK_Result,Off_STL_BLK,Off_L5_STL_BLK,AVG_STL_BLK_H2H,Def_STL_BLK,Def_L5_STL_BLK,STL_BLK_line,STL_BLK,STL_BLK_Diff,STL_BLK_Result
0,20251124_CLE_TOR,2025-11-24,CLE,1,NaN,NaN,Tyrese Proctor,PG,19.53,TOR,6.636364,7.6,NaN,20.636364,22.6,10.5,8,-2.5,U,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,None,2.000000,2.4,NaN,2.272727,3.4,1.5,0,-1.5,U,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None
1,20251124_POR_MIL,2025-11-24,MIL,1,NaN,NaN,Ryan Rollins,PG,30.45,POR,18.647059,22.0,NaN,24.000000,24.8,22.5,10,-12.5,U,5.941176,6.6,NaN,6.166667,7.0,6.5,7,0.5,O,4.187500,4.6,NaN,4.916667,4.6,4.5,5,0.5,O,22.588235,26.6,NaN,28.916667,29.4,27.5,15,-12.5,U,24.588235,28.6,NaN,30.166667,31.8,29.5,17,-12.5,U,9.882353,11.2,NaN,11.083333,11.6,11.5,12,0.5,O,28.529412,33.2,NaN,35.083333,36.4,34.5,22,-12.5,U,2.800000,3.6,NaN,2.833333,3.6,2.5,0,-2.5,U,1.928571,1.2,NaN,1.083333,1.0,1.5,1,-0.5,U,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,None
2,20251124_HOU_PHO,2025-11-24,HOU,0,NaN,NaN,Reed Sheppard,PG,26.90,PHO,13.923077,17.8,NaN,20.400000,19.0,17.5,7,-10.5,U,3.000000,2.8,NaN,6.400000,5.4,3.5,5,1.5,O,3.555556,3.2,NaN,4.533333,4.0,3.5,1,-2.5,U,16.384615,20.8,NaN,24.933333,23.0,21.5,8,-13.5,U,16.923077,20.6,NaN,26.800000,24.4,22.5,12,-10.5,U,5.461538,5.8,NaN,10.933333,9.4,7.5,6,-1.5,U,19.384615,23.6,NaN,31.333333,28.4,25.5,13,-12.5,U,3.166667,3.4,NaN,2.333333,1.8,3.5,1,-2.5,U,2.100000,2.0,NaN,1.333333,1.6,1.5,3,1.5,O,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,None
3,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Dennis Schroder,PG,24.30,MIN,13.562500,16.2,11.428571,21.545455,17.4,10.5,6,-4.5,U,5.823529,5.4,4.857143,6.545455,6.8,4.5,9,4.5,O,3.812500,4.4,2.857143,6.818182,7.0,2.5,5,2.5,O,17.375000,20.6,14.285714,28.363636,24.4,13.5,11,-2.5,U,18.588235,21.6,16.285714,28.090909,24.2,15.5,15,-0.5,U,9.411765,9.8,7.714286,13.363636,13.8,7.5,14,6.5,O,22.176471,26.0,19.142857,34.909091,31.2,18.5,20,1.5,O,2.454545,2.8,1.285714,2.545455,2.4,1.5,0,-1.5,U,1.363636,1.0,0.714286,1.000000,1.0,0.5,0,-0.5,U,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None
4,20251124_DET_IND,2025-11-24,IND,0,NaN,NaN,Andrew Nembhard,PG,36.92,DET,19.000000,21.2,7.875000,19.285714,18.0,16.5,12,-4.5,U,6.444444,6.4,3.625000,5.642857,4.2,6.5,6,-0.5,U,1.875000,2.0,2.000000,4.785714,3.6,2.5,1,-1.5,U,20.666667,23.2,9.875000,24.071429,21.6,19.5,13,-6.5,U,25.444444,27.6,11.500000,24.928571,22.2,22.5,18,-4.5,U,8.111111,8.4,5.625000,10.428571,7.8,8.5,7,-1.5,U,27.111111,29.6,13.500000,29.714286,25.8,25.5,19,-6.5,U,2.125000,2.0,0.375000,2.142857,1.8,1.5,2,0.5,O,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [15]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    print(f"==={stat}===")
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Spread, Total, Player, Pos, MP, Opp, 
                            Off_{stat}, Off_L5_{stat}, AVG_{stat}_H2H, 
                            Def_{stat}, Def_L5_{stat}, {stat}_line, {stat}, 
                            {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL AND {stat}_Result = 'O'
                            ORDER BY {stat}_Diff DESC
                            LIMIT 15""").fetchdf())

===PTS===


,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,PTS,PTS_Diff,PTS_Result
0,20251124_NYK_BRK,2025-11-24,BRK,0,NaN,NaN,Noah Clowney,PF,39.10,NYK,11.800000,15.6,6.833333,18.400000,16.8,13.5,31,17.5,O
1,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,DeMar DeRozan,PF,31.70,MIN,17.882353,14.8,24.500000,19.066667,17.4,17.5,33,15.5,O
2,20251124_HOU_PHO,2025-11-24,HOU,0,NaN,NaN,Aaron Holiday,PG,27.32,PHO,7.428571,9.0,4.714286,20.400000,19.0,7.5,22,14.5,O
3,20251124_MIN_SAC,2025-11-24,MIN,0,NaN,NaN,Anthony Edwards,SG,40.25,SAC,26.500000,25.6,28.222222,19.608696,18.2,28.5,43,14.5,O
4,20251124_CLE_TOR,2025-11-24,TOR,1,NaN,NaN,Brandon Ingram,SF,37.23,CLE,20.705882,21.2,17.600000,19.117647,15.0,22.5,37,14.5,O
5,20251124_POR_MIL,2025-11-24,POR,1,NaN,NaN,Jerami Grant,PF,30.92,MIL,18.562500,19.6,12.000000,18.312500,15.0,20.5,35,14.5,O
6,20251124_DEN_MEM,2025-11-24,DEN,0,NaN,NaN,Peyton Watson,SF,32.90,MEM,9.928571,14.6,11.428571,17.157895,14.0,12.5,27,14.5,O
7,20251124_DET_IND,2025-11-24,IND,0,NaN,NaN,Jarace Walker,PF,27.05,DET,9.687500,6.6,3.777778,16.333333,18.0,7.5,21,13.5,O
8,20251124_CHI_NOP,2025-11-24,CHI,0,NaN,NaN,Ayo Dosunmu,SG,32.17,NOP,15.571429,16.8,NaN,18.090909,13.2,14.5,28,13.5,O
9,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Keegan Murray,PF,44.40,MIN,15.000000,15.0,16.833333,19.066667,17.4,12.5,26,13.5,O


===AST===


,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,AST,AST_Diff,AST_Result
0,20251124_DEN_MEM,2025-11-24,DEN,0,NaN,NaN,Nikola Jokic,C,33.72,MEM,10.687500,10.2,8.600000,3.625000,4.2,10.5,16,5.5,O
1,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Dennis Schroder,PG,24.30,MIN,5.823529,5.4,4.857143,6.545455,6.8,4.5,9,4.5,O
2,20251124_MIN_SAC,2025-11-24,MIN,0,NaN,NaN,Mike Conley,PG,24.97,SAC,3.857143,3.0,6.222222,7.272727,6.8,2.5,5,2.5,O
3,20251124_HOU_PHO,2025-11-24,HOU,0,NaN,NaN,Amen Thompson,SF,36.85,PHO,5.000000,4.4,4.000000,2.909091,3.6,5.5,8,2.5,O
4,20251124_CLE_TOR,2025-11-24,CLE,1,NaN,NaN,Donovan Mitchell,SG,33.77,TOR,5.312500,5.0,4.571429,3.823529,3.4,5.5,8,2.5,O
5,20251124_DAL_MIA,2025-11-24,MIA,0,NaN,NaN,Pelle Larsson,SG,28.18,DAL,4.142857,5.0,NaN,3.923077,3.8,3.5,6,2.5,O
6,20251124_UTA_GSW,2025-11-24,UTA,1,NaN,NaN,Lauri Markkanen,PF,32.70,GSW,2.400000,2.2,NaN,2.357143,2.0,1.5,4,2.5,O
7,20251124_DEN_MEM,2025-11-24,MEM,0,NaN,NaN,Vince Williams Jr.,SG,23.92,DEN,5.000000,7.4,1.333333,3.500000,2.8,5.5,7,1.5,O
8,20251124_CLE_TOR,2025-11-24,CLE,1,NaN,NaN,Lonzo Ball,PG,34.05,TOR,5.071429,5.0,4.750000,5.454545,6.8,5.5,7,1.5,O
9,20251124_DEN_MEM,2025-11-24,DEN,0,NaN,NaN,Jamal Murray,PG,37.18,MEM,6.600000,8.4,6.333333,4.769231,5.4,6.5,8,1.5,O


===REB===


...

,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,PA,PA_Diff,PA_Result
0,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,DeMar DeRozan,PF,31.70,MIN,21.470588,19.0,29.333333,21.933333,20.2,21.5,37,15.5,O
1,20251124_POR_MIL,2025-11-24,POR,1,NaN,NaN,Jerami Grant,PF,30.92,MIL,20.937500,22.2,13.750000,22.562500,18.6,23.5,39,15.5,O
2,20251124_MIN_SAC,2025-11-24,MIN,0,NaN,NaN,Anthony Edwards,SG,40.25,SAC,30.000000,29.4,32.777778,23.956522,22.8,32.5,47,14.5,O
3,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Keegan Murray,PF,44.40,MIN,16.000000,16.0,18.000000,21.933333,20.2,14.5,28,13.5,O
4,20251124_DET_IND,2025-11-24,IND,0,NaN,NaN,Jarace Walker,PF,27.05,DET,12.312500,7.6,4.777778,18.916667,19.8,9.5,23,13.5,O
5,20251124_NYK_BRK,2025-11-24,NYK,0,NaN,NaN,Karl-Anthony Towns,C,32.97,BRK,25.066667,27.4,25.833333,22.100000,20.6,26.5,40,13.5,O
6,20251124_CLE_TOR,2025-11-24,TOR,1,NaN,NaN,Brandon Ingram,SF,37.23,CLE,24.705882,25.2,22.600000,23.000000,18.8,26.5,39,12.5,O
7,20251124_CHI_NOP,2025-11-24,CHI,0,NaN,NaN,Ayo Dosunmu,SG,32.17,NOP,18.642857,20.0,NaN,21.318182,16.2,17.5,29,11.5,O
8,20251124_HOU_PHO,2025-11-24,HOU,0,NaN,NaN,Amen Thompson,SF,36.85,PHO,22.153846,21.0,12.000000,18.363636,19.6,24.5,36,11.5,O
9,20251124_UTA_GSW,2025-11-24,UTA,1,NaN,NaN,Ace Bailey,SF,29.48,GSW,11.647059,14.8,NaN,19.071429,22.8,12.5,23,10.5,O


===RA===


,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,RA,RA_Diff,RA_Result
0,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Keegan Murray,PF,44.40,MIN,5.000000,5.000000,8.500000,8.400000,8.6,6.5,16,9.5,O
1,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Dennis Schroder,PG,24.30,MIN,9.411765,9.800000,7.714286,13.363636,13.8,7.5,14,6.5,O
2,20251124_NYK_BRK,2025-11-24,NYK,0,NaN,NaN,Josh Hart,SF,37.67,BRK,11.214286,12.600000,14.000000,8.833333,9.6,14.5,19,4.5,O
3,20251124_CLE_TOR,2025-11-24,CLE,1,NaN,NaN,Lonzo Ball,PG,34.05,TOR,9.214286,9.600000,8.750000,9.000000,10.6,9.5,14,4.5,O
4,20251124_CLE_TOR,2025-11-24,TOR,1,NaN,NaN,Scottie Barnes,PF,38.30,CLE,12.588235,12.200000,15.111111,13.444444,10.0,13.5,17,3.5,O
5,20251124_UTA_GSW,2025-11-24,UTA,1,NaN,NaN,Keyonte George,PG,37.75,GSW,11.235294,11.400000,6.714286,13.111111,13.6,9.5,13,3.5,O
6,20251124_CLE_TOR,2025-11-24,TOR,1,NaN,NaN,Immanuel Quickley,PG,33.02,CLE,10.705882,11.000000,7.000000,10.571429,7.8,10.5,13,2.5,O
7,20251124_CHI_NOP,2025-11-24,CHI,0,NaN,NaN,Coby White,SG,35.27,NOP,9.666667,9.666667,11.500000,7.227273,7.2,8.5,11,2.5,O
8,20251124_MIN_SAC,2025-11-24,MIN,0,NaN,NaN,Julius Randle,PF,38.68,SAC,13.062500,12.400000,11.400000,9.307692,8.8,12.5,15,2.5,O
9,20251124_DET_IND,2025-11-24,DET,0,NaN,NaN,Cade Cunningham,PG,35.80,IND,15.538462,16.600000,13.375000,11.142857,10.8,14.5,17,2.5,O


===PRA===


,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,PRA,PRA_Diff,PRA_Result
0,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Keegan Murray,PF,44.40,MIN,20.000000,20.0,25.333333,27.466667,26.0,19.5,42,22.5,O
1,20251124_POR_MIL,2025-11-24,POR,1,NaN,NaN,Jerami Grant,PF,30.92,MIL,24.375000,26.8,15.000000,28.875000,25.2,27.5,43,15.5,O
2,20251124_MIN_SAC,2025-11-24,MIN,0,NaN,NaN,Anthony Edwards,SG,40.25,SAC,34.333333,34.0,37.777778,28.565217,26.2,38.5,54,15.5,O
3,20251124_DEN_MEM,2025-11-24,DEN,0,NaN,NaN,Peyton Watson,SF,32.90,MEM,14.937500,23.2,17.714286,26.210526,21.8,20.5,35,14.5,O
4,20251124_DET_IND,2025-11-24,IND,0,NaN,NaN,Jarace Walker,PF,27.05,DET,17.375000,12.2,6.444444,25.416667,25.8,14.5,29,14.5,O
5,20251124_DEN_MEM,2025-11-24,MEM,0,NaN,NaN,Jaylen Wells,SG,33.40,DEN,14.705882,13.0,NaN,24.400000,23.8,16.5,30,13.5,O
6,20251124_CLE_TOR,2025-11-24,TOR,1,NaN,NaN,Brandon Ingram,SF,37.23,CLE,30.470588,30.2,28.600000,28.941176,24.2,32.5,46,13.5,O
7,20251124_NYK_BRK,2025-11-24,NYK,0,NaN,NaN,Karl-Anthony Towns,C,32.97,BRK,37.533333,39.4,36.166667,32.400000,30.6,39.5,52,12.5,O
8,20251124_UTA_GSW,2025-11-24,UTA,1,NaN,NaN,Keyonte George,PG,37.75,GSW,34.058824,38.0,25.857143,33.722222,32.4,29.5,41,11.5,O
9,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,DeMar DeRozan,PF,31.70,MIN,24.764706,22.0,33.333333,27.466667,26.0,25.5,37,11.5,O


In [16]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    print(f"==={stat}===")
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Spread, Total, Player, Pos, MP, Opp, 
                            Off_{stat}, Off_L5_{stat}, AVG_{stat}_H2H, 
                            Def_{stat}, Def_L5_{stat}, {stat}_line, {stat}, 
                            {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL AND {stat}_Result = 'U'
                            ORDER BY {stat}_Diff
                            LIMIT 15""").fetchdf())

===PTS===


,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_PTS,Off_L5_PTS,AVG_PTS_H2H,Def_PTS,Def_L5_PTS,PTS_line,PTS,PTS_Diff,PTS_Result
0,20251124_CLE_TOR,2025-11-24,CLE,1,NaN,NaN,Donovan Mitchell,SG,33.77,TOR,30.750000,31.40,21.428571,15.000000,20.4,30.5,17,-13.5,U
1,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Zach LaVine,SG,31.82,MIN,21.400000,16.40,NaN,16.562500,14.6,20.5,7,-13.5,U
2,20251124_DEN_MEM,2025-11-24,DEN,0,NaN,NaN,Nikola Jokic,C,33.72,MEM,30.375000,33.80,23.600000,18.500000,18.0,30.5,17,-13.5,U
3,20251124_DAL_MIA,2025-11-24,DAL,0,NaN,NaN,D'Angelo Russell,PG,9.77,MIA,12.625000,15.20,12.800000,23.000000,20.6,12.5,0,-12.5,U
4,20251124_POR_MIL,2025-11-24,MIL,1,NaN,NaN,Ryan Rollins,PG,30.45,POR,18.647059,22.00,NaN,24.000000,24.8,22.5,10,-12.5,U
5,20251124_HOU_PHO,2025-11-24,HOU,0,NaN,NaN,Reed Sheppard,PG,26.90,PHO,13.923077,17.80,NaN,20.400000,19.0,17.5,7,-10.5,U
6,20251124_NYK_BRK,2025-11-24,BRK,0,NaN,NaN,Ziaire Williams,SF,0.00,NYK,9.214286,9.40,6.500000,17.416667,16.6,10.5,0,-10.5,U
7,20251124_DEN_MEM,2025-11-24,MEM,0,NaN,NaN,Zach Edey,C,5.68,DEN,12.250000,12.25,NaN,16.222222,17.8,11.5,2,-9.5,U
8,20251124_CHI_NOP,2025-11-24,NOP,0,NaN,NaN,Derik Queen,C,16.95,CHI,12.647059,17.60,NaN,20.500000,21.4,17.5,8,-9.5,U
9,20251124_HOU_PHO,2025-11-24,PHO,0,NaN,NaN,Devin Booker,SG,37.78,HOU,26.882353,23.80,25.166667,14.800000,18.0,27.5,18,-9.5,U


===AST===


,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_AST,Off_L5_AST,AVG_AST_H2H,Def_AST,Def_L5_AST,AST_line,AST,AST_Diff,AST_Result
0,20251124_DET_IND,2025-11-24,DET,0,NaN,NaN,Cade Cunningham,PG,35.80,IND,9.923077,10.4,8.375000,6.000000,5.8,9.5,6,-3.5,U
1,20251124_HOU_PHO,2025-11-24,PHO,0,NaN,NaN,Collin Gillespie,PG,29.30,HOU,5.117647,4.8,2.000000,7.600000,7.6,5.5,2,-3.5,U
2,20251124_UTA_GSW,2025-11-24,GSW,1,NaN,NaN,Stephen Curry,PG,29.05,UTA,4.384615,4.6,6.200000,7.571429,8.4,5.5,2,-3.5,U
3,20251124_NYK_BRK,2025-11-24,NYK,0,NaN,NaN,Jalen Brunson,PG,32.73,BRK,6.769231,7.8,7.375000,5.900000,4.8,6.5,3,-3.5,U
4,20251124_HOU_PHO,2025-11-24,PHO,0,NaN,NaN,Devin Booker,SG,37.78,HOU,7.058824,7.2,4.000000,2.900000,2.4,7.5,5,-2.5,U
5,20251124_POR_MIL,2025-11-24,MIL,1,NaN,NaN,Cole Anthony,PG,17.68,POR,5.125000,5.8,1.250000,6.166667,7.0,4.5,2,-2.5,U
6,20251124_CHI_NOP,2025-11-24,CHI,0,NaN,NaN,Josh Giddey,PG,35.05,NOP,9.714286,10.4,4.666667,7.250000,6.6,8.5,6,-2.5,U
7,20251124_DAL_MIA,2025-11-24,DAL,0,NaN,NaN,D'Angelo Russell,PG,9.77,MIA,5.000000,5.2,3.400000,8.090909,8.8,4.5,2,-2.5,U
8,20251124_UTA_GSW,2025-11-24,UTA,1,NaN,NaN,Isaiah Collier,PG,21.18,GSW,6.111111,5.4,NaN,7.500000,7.2,5.5,3,-2.5,U
9,20251124_CHI_NOP,2025-11-24,CHI,0,NaN,NaN,Ayo Dosunmu,SG,32.17,NOP,3.071429,3.2,NaN,3.227273,3.0,3.5,1,-2.5,U


===REB===


...

,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_PA,Off_L5_PA,AVG_PA_H2H,Def_PA,Def_L5_PA,PA_line,PA,PA_Diff,PA_Result
0,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Zach LaVine,SG,31.82,MIN,23.666667,19.0,NaN,21.000000,18.6,23.5,8,-15.5,U
1,20251124_DAL_MIA,2025-11-24,DAL,0,NaN,NaN,D'Angelo Russell,PG,9.77,MIA,17.625000,20.4,16.200000,31.090909,29.4,17.5,2,-15.5,U
2,20251124_POR_MIL,2025-11-24,MIL,1,NaN,NaN,Ryan Rollins,PG,30.45,POR,24.588235,28.6,NaN,30.166667,31.8,29.5,17,-12.5,U
3,20251124_HOU_PHO,2025-11-24,PHO,0,NaN,NaN,Devin Booker,SG,37.78,HOU,33.941176,31.0,29.166667,17.700000,20.4,34.5,23,-11.5,U
4,20251124_NYK_BRK,2025-11-24,BRK,0,NaN,NaN,Michael Porter Jr.,SF,27.48,NYK,28.000000,32.6,18.000000,20.916667,20.6,27.5,16,-11.5,U
5,20251124_CHI_NOP,2025-11-24,NOP,0,NaN,NaN,Derik Queen,C,16.95,CHI,15.882353,23.0,NaN,24.857143,26.8,22.5,12,-10.5,U
6,20251124_CLE_TOR,2025-11-24,CLE,1,NaN,NaN,Donovan Mitchell,SG,33.77,TOR,36.062500,36.4,26.000000,18.823529,23.8,35.5,25,-10.5,U
7,20251124_DEN_MEM,2025-11-24,MEM,0,NaN,NaN,Cedric Coward,SG,26.67,DEN,17.117647,16.0,NaN,20.900000,20.4,17.5,7,-10.5,U
8,20251124_HOU_PHO,2025-11-24,HOU,0,NaN,NaN,Reed Sheppard,PG,26.90,PHO,16.923077,20.6,NaN,26.800000,24.4,22.5,12,-10.5,U
9,20251124_UTA_GSW,2025-11-24,UTA,1,NaN,NaN,Isaiah Collier,PG,21.18,GSW,16.000000,15.4,NaN,28.111111,26.0,14.5,5,-9.5,U


===RA===


,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_RA,Off_L5_RA,AVG_RA_H2H,Def_RA,Def_L5_RA,RA_line,RA,RA_Diff,RA_Result
0,20251124_CHI_NOP,2025-11-24,NOP,0,NaN,NaN,Zion Williamson,PF,29.62,CHI,10.428571,8.4,NaN,10.214286,8.8,10.5,3,-7.5,U
1,20251124_HOU_PHO,2025-11-24,HOU,0,NaN,NaN,Alperen Sengun,C,34.70,PHO,17.230769,17.4,11.714286,14.400000,13.2,17.5,10,-7.5,U
2,20251124_NYK_BRK,2025-11-24,BRK,0,NaN,NaN,Michael Porter Jr.,SF,27.48,NYK,10.800000,12.4,6.600000,8.583333,7.8,9.5,4,-5.5,U
3,20251124_POR_MIL,2025-11-24,POR,1,NaN,NaN,Deni Avdija,SF,31.38,MIL,12.000000,14.0,11.600000,9.352941,5.6,14.5,9,-5.5,U
4,20251124_CLE_TOR,2025-11-24,CLE,1,NaN,NaN,Evan Mobley,PF,31.18,TOR,12.764706,12.6,14.500000,10.764706,7.2,14.5,9,-5.5,U
5,20251124_CHI_NOP,2025-11-24,NOP,0,NaN,NaN,Derik Queen,C,16.95,CHI,9.823529,13.4,NaN,17.000000,19.6,13.5,9,-4.5,U
6,20251124_CHI_NOP,2025-11-24,CHI,0,NaN,NaN,Josh Giddey,PG,35.05,NOP,19.785714,21.4,11.444444,11.375000,9.6,17.5,13,-4.5,U
7,20251124_NYK_BRK,2025-11-24,NYK,0,NaN,NaN,Jordan Clarkson,SG,20.07,BRK,4.071429,5.4,NaN,7.647059,6.6,5.5,1,-4.5,U
8,20251124_NYK_BRK,2025-11-24,NYK,0,NaN,NaN,Jalen Brunson,PG,32.73,BRK,10.230769,10.6,10.625000,10.900000,11.8,10.5,6,-4.5,U
9,20251124_DAL_MIA,2025-11-24,MIA,0,NaN,NaN,Bam Adebayo,C,29.33,DAL,11.272727,10.2,8.250000,12.923077,15.4,11.5,7,-4.5,U


===PRA===


,game_id,Date,Team,B2B,Spread,Total,Player,Pos,MP,Opp,Off_PRA,Off_L5_PRA,AVG_PRA_H2H,Def_PRA,Def_L5_PRA,PRA_line,PRA,PRA_Diff,PRA_Result
0,20251124_DEN_MEM,2025-11-24,MEM,0,NaN,NaN,Zach Edey,C,5.68,DEN,22.750000,22.75,NaN,28.111111,29.4,22.5,3,-19.5,U
1,20251124_MIN_SAC,2025-11-24,SAC,0,NaN,NaN,Zach LaVine,SG,31.82,MIN,27.000000,22.20,NaN,24.687500,22.8,27.5,11,-16.5,U
2,20251124_DAL_MIA,2025-11-24,DAL,0,NaN,NaN,D'Angelo Russell,PG,9.77,MIA,20.562500,23.40,17.800000,38.454545,36.6,19.5,3,-16.5,U
3,20251124_NYK_BRK,2025-11-24,BRK,0,NaN,NaN,Michael Porter Jr.,SF,27.48,NYK,35.600000,40.60,24.200000,26.000000,24.4,34.5,20,-14.5,U
4,20251124_CLE_TOR,2025-11-24,CLE,1,NaN,NaN,Donovan Mitchell,SG,33.77,TOR,41.062500,42.40,30.857143,22.470588,27.6,40.5,26,-14.5,U
5,20251124_HOU_PHO,2025-11-24,HOU,0,NaN,NaN,Alperen Sengun,C,34.70,PHO,38.692308,40.00,29.285714,30.500000,31.4,42.5,28,-14.5,U
6,20251124_CHI_NOP,2025-11-24,NOP,0,NaN,NaN,Derik Queen,C,16.95,CHI,22.470588,31.00,NaN,37.500000,41.0,30.5,17,-13.5,U
7,20251124_HOU_PHO,2025-11-24,PHO,0,NaN,NaN,Devin Booker,SG,37.78,HOU,38.117647,35.20,33.166667,21.600000,24.2,38.5,26,-12.5,U
8,20251124_POR_MIL,2025-11-24,MIL,1,NaN,NaN,Ryan Rollins,PG,30.45,POR,28.529412,33.20,NaN,35.083333,36.4,34.5,22,-12.5,U
9,20251124_DEN_MEM,2025-11-24,MEM,0,NaN,NaN,Cedric Coward,SG,26.67,DEN,23.117647,22.20,NaN,24.400000,23.8,22.5,10,-12.5,U
